In [1]:
path_data = '/data'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore")
import os

import pandas as pd

import torch
from pkg_resources import packaging
from PIL import Image
import tqdm
from pathlib import Path
print("Torch version:", torch.__version__)

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

plt.style.use('ggplot')

Torch version: 2.2.1+cu121


In [3]:
data_unfiltered = pd.read_csv("/".join([path_data, "images_v2.csv"]))
#display(data_unfiltered)

In [4]:
len(data_unfiltered)

38479

### Remove subjective topics

In [5]:
subjective_topics = ['Favorite home decorations', 'Favourite item in kitchen', 'Favourite sports clubs', 'How the most loved item is used', 'icons', 'Idols', 'Latest furniture bought', ' looking over the shoulder', 'Most loved item', 'Most loved toy', 'Most played songs on the radio', 'Music idol', 'Next big thing you are planning to buy', 'Playing with most loved toy', 'Thing I dream about having', 'Things I wish I had', 'Using most loved item', 'Youth culture', 'What I wish I could buy']

In [6]:
len(subjective_topics)

19

In [7]:
unique_topics = data_unfiltered['topics'].unique()

In [8]:
Subjective_topic_list = []

for topic in unique_topics:
    topic_ = topic.replace('/ ', ', ')
    topic_split = topic_.split(',')
    for split in topic_split:
        if split in subjective_topics:
            Subjective_topic_list.append(topic)

In [9]:
data = data_unfiltered[~data_unfiltered['topics'].isin(Subjective_topic_list)]
display(data)

,id,country.name,country.id,region.id,type,imageRelPath,topics,place,income
1,5d4bf31ccf0b3a0f3f359814,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359814/5d4bf31ccf0b3a...,Family snapshots,butoyi,26.994581
2,5d4bf31ccf0b3a0f3f35982a,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982a/5d4bf31ccf0b3a...,Cutlery,butoyi,26.994581
3,5d4bf31ccf0b3a0f3f35982e,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f35982e/5d4bf31ccf0b3a...,Family,butoyi,26.994581
4,5d4bf31ccf0b3a0f3f359830,Burundi,bi,af,image,assets/5d4bf31ccf0b3a0f3f359830/5d4bf31ccf0b3a...,Place where eating dinner,butoyi,26.994581
5,5d4bf31dcf0b3a0f3f35983c,Burundi,bi,af,image,assets/5d4bf31dcf0b3a0f3f35983c/5d4bf31dcf0b3a...,Plate of food,butoyi,26.994581
...,...,...,...,...,...,...,...,...,...
38474,5ec4f5513f62767d97a47324,France,fr,eu,image,assets/5ec4f5513f62767d97a47324/5ec4f5513f6276...,Bed,larriere,19671.000000
38475,5ec4f5513f62767d97a47325,France,fr,eu,image,assets/5ec4f5513f62767d97a47325/5ec4f5513f6276...,Bathroom/Toilet,larriere,19671.000000
38476,5ec4f5523f62767d97a47327,France,fr,eu,image,assets/5ec4f5523f62767d97a47327/5ec4f5523f6276...,Armchair,larriere,19671.000000
38477,5ec4f5523f62767d97a47328,France,fr,eu,image,assets/5ec4f5523f62767d97a47328/5ec4f5523f6276...,Armchair,larriere,19671.000000


In [10]:
len(data)

36754

### Split topics

In [11]:
from collections import Counter
list_topics = list(data['topics'])


separate_topics = [t.lower().strip() for topic in list_topics for t in topic.split(",") ]


print(len(separate_topics))
#print(Counter(separate_topics))
set_topics = list(set(separate_topics))
print(len(set_topics))

45692
270


In [12]:
len(separate_topics)

45692

In [13]:
# map each topic to list of corresponding images
dict_topic2img = {}
for list_topics, image_path in zip(data['topics'], data['imageRelPath']):
    for topic in list_topics.split(","):
        topic = topic.lower().strip()
        if topic not in dict_topic2img:
            dict_topic2img[topic] = set() #### here a set was used in place of list to avoid duplicate where keyword appears twice in a topic
        dict_topic2img[topic].add(image_path)

print(len(dict_topic2img))

270


In [14]:
#sanity checks 

ground_truth_counts = [len(dict_topic2img[i]) for i in dict_topic2img]
print(sum(ground_truth_counts))

45616


In [15]:
topics = dict_topic2img.keys()

# Create prompts for the text encoder

In [16]:
temp = "This is a photo of "
English_topics = [temp + topic for topic in topics]

In [17]:
#English_topics

In [18]:
len(English_topics)

270

In [19]:
len(set(English_topics))

270

In [20]:
topic2english = {}
for topic, prompt in zip(topics, English_topics):
    topic2english[topic] = prompt

In [21]:
english_dict_topic2imgGT = {}

for topic in topics:
    key = topic2english[topic]
    english_dict_topic2imgGT[key] = dict_topic2img[topic]
print(len(english_dict_topic2imgGT))

270


### Make Language prompts

In [73]:
selected_languages = ['German', 'Spanish', 'Portuguese', 'French', 'Chinese', 'Czech', 'Danish', 'Arabic', 'Hindi', 'Indonesian', 'Farsi_Persian', 'Italian', 'Russian', 'Mongolian', 'Burmese', 'Dutch', 'Urdu', 'Romanian', 'Serbian', 'Korean', 'Swedish', 'Thai', 'Turkish', 'Ukrainian', 'Vietnamese', 'Bengali', 'Khmer', 'Oromo', 'Ewe', 'Creole', 'Swahili', 'Nepali', 'Hausa', 'Kyrgz', 'Tagalog', 'Kinyarwanda', 'Somali', 'Zulu', 'Sinhala', 'Shona']

In [70]:
selected_languages_code = ['deu_Latn', 'spa_Latn', 'por_Latn', 'fra_Latn', 'zho_Hans', 'ces_Latn', 'dan_Latn', 'arb_Arab', 'hin_Deva', 'ind_Latn', 'pes_Arab', 'ita_Latn', 'rus_Cyrl', 'khk_Cyrl', 'mya_Mymr', 'nld_Latn', 'urd_Arab', 'ron_Latn', 'srp_Cyrl', 'kor_Hang', 'swe_Latn', 'tha_Thai', 'tur_Latn', 'ukr_Cyrl', 'vie_Latn', 'ben_Beng', 'khm_Khmr', 'gaz_Latn', 'ewe_Latn', 'hat_Latn', 'swh_Latn', 'npi_Deva', 'hau_Latn', 'kir_Cyrl', 'tgl_Latn', 'kin_Latn', 'som_Latn', 'zul_Latn', 'sin_Sinh', 'sna_Latn']

In [74]:
len(selected_languages)

40

In [27]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")


In [36]:
list_eng2translated_text_dicts = []
list_eng2translated_text_dicts_merged = []
list_dict_lang2translation = []
#dict_lang2translation


#list_lang2translated_list = []


for lang_code, lang in tqdm.tqdm(zip(selected_languages_code, selected_languages)):
    print(lang)
    
    dict_eng2translation = {}
    dict_lang2translation = {}
    dict_eng2translation_merged = {}

    txt_list = English_topics
    inputs = tokenizer(txt_list, return_tensors="pt", padding=True)
    translated_tokens = model.generate(
    **inputs, forced_bos_token_id=tokenizer.lang_code_to_id[lang_code])
    
    translations = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)
    dict_lang2translation[lang] = translations
    
    for eng, trans in zip(English_topics, translations):
        dict_eng2translation[eng] = trans
        dict_eng2translation_merged[eng]= lang_code + ' ; ' + eng + ' ; ' + trans
    
    list_dict_lang2translation.append(dict_lang2translation)
    list_eng2translated_text_dicts.append(dict_eng2translation)
    list_eng2translated_text_dicts_merged.append(dict_eng2translation_merged)

0it [00:00, ?it/s]

German


1it [00:29, 29.62s/it]

Spanish


2it [00:55, 27.42s/it]

...

38it [57:39, 190.61s/it]

Sinhala


39it [1:03:19, 235.28s/it]

Swahili


40it [1:03:47, 95.69s/it] 


In [37]:
len(list_dict_lang2translation)

40

In [81]:
import pickle
f = open("list_eng2translated_text_dicts.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(list_eng2translated_text_dicts,f)

# close file
f.close()

In [100]:
f = open("list_dict_lang2translation.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(list_dict_lang2translation,f)

# close file
f.close()

In [83]:
f = open("list_eng2translated_text_dicts_merged.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(list_eng2translated_text_dicts_merged,f)

# close file
f.close()

In [99]:
#list_dict_lang2translation[39]['Shona']

['Mufananidzo uyu unoratidza mhuri',
 'Uyu mufananidzo wemidziyo',
 'Uyu mufananidzo wemhuri',
 'Uyu mufananidzo wenzvimbo apo kudya kwemanheru',
 'Uyu mufananidzo wezvokudya',
 'Uyu mufananidzo patafura ine zvokudya',
 'Uyu mufananidzo wokutora mvura',
 'Uyu mufananidzo wechitofu / hob',
 'Uyu mufananidzo wechiedza chiri mukicheni',
 'Uyu mufananidzo wemvura yokunwa',
 'Uyu mufananidzo wetsvina yekicheni',
 'Uyu mufananidzo wokugara wega mumba mokuchera',
 'Uyu mufananidzo weimba yapashure',
 'Uyu mufananidzo pamukova wokutanga',
 'Uyu mufananidzo wemba',
 'Uyu mufananidzo tsime Cool',
 'Uyu mufananidzo wemugwagwa',
 'Uyu mufananidzo wemeno',
 'Uyu mufananidzo ruoko shure',
 'Uyu mufananidzo woruoko rwemaoko',
 'Uyu mufananidzo wemhuka',
 'Uyu mufananidzo wokurara',
 'Uyu mufananidzo wezviyo',
 'Ichi chifananidzo chemucheka',
 'Uyu mufananidzo wechigaro',
 'Uyu mufananidzo wepamberi',
 'Uyu mufananidzo wekicheni',
 'Ichi chifananidzo chemabhuku',
 'Uyu mufananidzo wezvishuwo zvezuva n

In [41]:
len(list_eng2translated_text_dicts_merged[5].values())

270

### Load translated topics

In [43]:
import pickle
with open("list_eng2translated_text_dicts.pkl", 'rb') as f:
    list_eng2translated_text_dicts = pickle.load(f)

In [44]:
with open("list_dict_lang2translation.pkl", 'rb') as f:
    dict_lang2translation = pickle.load(f)

In [45]:
dict_lang2translation[3]

{'French': ["C'est une photo de photos de famille.",
  "C' est une photo de la cuisine.",
  "C'est une photo de famille.",
  "C'est une photo d'un endroit où on mange le dîner.",
  "C'est une photo de la assiette de nourriture.",
  "C'est une photo de table avec de la nourriture.",
  "C'est une photo de l'eau",
  "C'est une photo du four/hô",
  "C'est une photo de la source lumineuse dans la cuisine.",
  "C'est une photo d'eau potable.",
  "C'est une photo du lavabo de cuisine.",
  "C'est une photo de la vie privée de la salle de bain.",
  "C'est une photo de la cour.",
  "C'est une photo de la porte d'entrée.",
  "C'est une photo de chez moi.",
  "C'est une photo de source de cool",
  "C'est une photo de la vue de la rue.",
  "C'est une photo de dents.",
  "C'est une photo de la main en arrière.",
  "C'est une photo de la paume de main.",
  "C'est une photo d'animal de compagnie.",
  "C'est une photo de la chambre.",
  "C'est une photo de grains.",
  "C' est une photo de l' armoire.",

In [46]:
len(list_eng2translated_text_dicts)

40

In [47]:
len(list_eng2translated_text_dicts[0].values())

270

In [48]:
len(set(list_eng2translated_text_dicts_merged[0].values()))

270

### Get ground truth

In [49]:
topic2english = {}
for topic, prompt in zip(topics, English_topics):
    topic2english[topic] = prompt

In [50]:
#topic2english

In [51]:
english_dict_topic2imgGT = {}

for topic in topics:
    key = topic2english[topic]
    english_dict_topic2imgGT[key] = dict_topic2img[topic]
print(len(english_dict_topic2imgGT))

270


In [52]:
len(english_dict_topic2imgGT['This is a photo of spices'])

419

In [53]:
with open("list_eng2translated_text_dicts_merged.pkl", 'rb') as f:
    list_eng2translated_text_dicts_merged = pickle.load(f)

In [54]:
list_dict_trans_prompt2img = []

for lang_dict in tqdm.tqdm(list_eng2translated_text_dicts_merged):
    dict_trans_prompt2img = {}
    for eng_prompt in lang_dict.keys():
        trans = lang_dict[eng_prompt]
        dict_trans_prompt2img[trans] = english_dict_topic2imgGT[eng_prompt]
        
    list_dict_trans_prompt2img.append(dict_trans_prompt2img)

100%|██████████| 40/40 [00:00<00:00, 11953.84it/s]


In [76]:
list(list_eng2translated_text_dicts_merged[39].values())[0].split(';')[0].strip()

'sna_Latn'

In [77]:
len([t.split(';')[-1].strip() for t in list_eng2translated_text_dicts_merged[0].values()])

270

### Load img embeddings

In [60]:
import pickle
with open("nllb_img_embedding.pkl", 'rb') as f:
    img_embedding = pickle.load(f)

In [61]:
img_embedding.shape

torch.Size([36754, 1152])

## Cosine sim for lang_prompts and imgs

In [62]:
from sentence_transformers import SentenceTransformer, util
from open_clip import get_tokenizer

from open_clip import create_model_from_pretrained
from PIL import Image, ImageFile
import timm
#vit_so400m_p

In [63]:
# Load model directly
from transformers import AutoProcessor, AutoModel

model, transform = create_model_from_pretrained("nllb-clip-large-siglip", "v1", device="cuda")
model = model.eval()

In [64]:
from open_clip import get_tokenizer

tokenizer = get_tokenizer("nllb-clip-large-siglip")

In [66]:
list_language_score_dicts = []
for lang_prompt in tqdm.tqdm(list_eng2translated_text_dicts_merged):
    text = [t.split(';')[-1].strip() for t in lang_prompt.values()]
    #text = list(lang_prompt.values())
    text_langs = [t.split(';')[0].strip() for t in lang_prompt.values()] 
    
    text_inputs = []
    for i in range(len(text)):
        tokenizer.set_language(text_langs[0])
        text_inputs.append(tokenizer(text[i]))

    text_inputs = torch.stack(text_inputs).squeeze(1)
    text_inputs = text_inputs.to("cuda")
    
    with torch.no_grad():
        text_features = model.encode_text(text_inputs)
        text_features /= text_features.norm(dim=-1, keepdim=True)
    
    #lang_text_emb = text_model.encode(text, convert_to_tensor=True, show_progress_bar=False)
    cossim = util.cos_sim(text_features, img_embedding)
    cos_sim = cossim.cpu().numpy()
    
    dict_trans_topic2score = {}
    for i, topic in enumerate(lang_prompt.values()):
        #print(i, topic)
        clip_scores = cos_sim[i] # clip scores between topic and images from data['imageRelPath']
        
        list_scores_img = list(zip(clip_scores, data['imageRelPath'])) # ******
        dict_trans_topic2score[topic] = sorted(list_scores_img, key = lambda x: x[0], reverse=True)
        
        
    list_language_score_dicts.append(dict_trans_topic2score)
    print(len(dict_trans_topic2score))

  2%|▎         | 1/40 [00:12<07:55, 12.19s/it]

270


  5%|▌         | 2/40 [00:19<05:43,  9.03s/it]

270


  8%|▊         | 3/40 [00:25<04:56,  8.02s/it]

...

270


 98%|█████████▊| 39/40 [04:43<00:07,  7.15s/it]

270


100%|██████████| 40/40 [04:50<00:00,  7.27s/it]

270


### Score Processing

In [105]:
## Trial for the first _dict  (recall and precision and F1 scores are equal)

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

all_images = list(data['imageRelPath'])
list_dict_topic2FNimg = []
list_dict_topic2F1 = []

for lang_dict in tqdm.tqdm(list_language_score_dicts[:1]):
    
    dict_topic2FNimg = {}
    dict_topic2F1 = {}
    
    for topic in tqdm.tqdm(list(lang_dict.keys())):
        print(topic)
        dict_lang_topic2img = next(item for item in list_dict_trans_prompt2img if topic in item.keys())
        count_pos = len(dict_lang_topic2img[topic])
        print(count_pos)
        
        GT_pos_images = dict_lang_topic2img[topic]
        Pred_pos_images = [data[1] for data in lang_dict[topic][:count_pos]]
        print(len(Pred_pos_images))

        GT_labels = [1 if img in GT_pos_images else 0 for img in all_images]
        Pred_labels = [1 if img in Pred_pos_images else 0 for img in all_images]
        
        
        accuracy = round(accuracy_score(GT_labels, Pred_labels) * 100, 2)
        precision = round(precision_score(GT_labels, Pred_labels) * 100, 2)
        recall = round(recall_score(GT_labels, Pred_labels) * 100, 2)
        f1 = round(f1_score(GT_labels, Pred_labels) * 100, 2)
        
        print(f"Acc: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}")
        
        dict_topic2F1[topic] = f1 
        list_dict_topic2F1.append(dict_topic2F1)
        
        
        # What are the forgotten items (FN) 
        images_FN = []
        for idx, img in enumerate(all_images):
            label_GT = GT_labels[idx]
            label_Pred = Pred_labels[idx]
            #FN
            if label_GT == 1 and label_Pred == 0:
                images_FN.append(img) 
        
        dict_topic2FNimg[topic] = images_FN
        
        list_dict_topic2FNimg.append(dict_topic2FNimg)
        

  0%|          | 1/270 [00:00<00:56,  4.80it/s]

deu_Latn ; This is a photo of family snapshots ; Das ist ein Foto von Familienbildern.
102
102
Acc: 99.58, Recall: 24.51, Precision: 24.51, F1: 24.51
deu_Latn ; This is a photo of cutlery ; Das ist ein Foto von der Kühlerei.
301
301



  1%|          | 2/270 [00:00<01:12,  3.71it/s]

Acc: 98.36, Recall: 0.0, Precision: 0.0, F1: 0.0
deu_Latn ; This is a photo of family ; Das ist ein Foto von der Familie.
518
518



  1%|          | 3/270 [00:00<01:33,  2.86it/s]

...

Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0
deu_Latn ; This is a photo of baking sheets ; Das ist ein Foto von Bäckchen.
1
1
Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0
deu_Latn ; This is a photo of baking tables ; Das ist ein Foto von Backtischen.
1
1



 99%|█████████▉| 268/270 [01:04<00:00,  6.78it/s]

Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0
deu_Latn ; This is a photo of electricity wires ; Das ist ein Foto von Stromleitungen.
1
1
Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0
deu_Latn ; This is a photo of meat markets ; Das ist ein Foto von Fleischmärkten.
1
1



100%|██████████| 1/1 [01:04<00:00, 64.66s/it]

Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0
deu_Latn ; This is a photo of vegetable markets ; Das ist ein Foto von Gemüsemärkten.
1
1
Acc: 99.99, Recall: 0.0, Precision: 0.0, F1: 0.0


In [67]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

all_images = list(data['imageRelPath'])
list_dict_topic2FNimg = []
list_dict_topic2F1 = []

for lang_dict in tqdm.tqdm(list_language_score_dicts):
    
    dict_topic2FNimg = {}
    dict_topic2F1 = {}
    
    for topic in list(lang_dict.keys()):
        #print(topic)
        dict_lang_topic2img = next(item for item in list_dict_trans_prompt2img if topic in item.keys())
        count_pos = len(dict_lang_topic2img[topic])
        #print(count_pos)
        
        GT_pos_images = dict_lang_topic2img[topic]
        Pred_pos_images = [data[1] for data in lang_dict[topic][:count_pos]]
        #print(len(Pred_pos_images))

        GT_labels = [1 if img in GT_pos_images else 0 for img in all_images]
        Pred_labels = [1 if img in Pred_pos_images else 0 for img in all_images]
        
        
        accuracy = round(accuracy_score(GT_labels, Pred_labels) * 100, 2)
        precision = round(precision_score(GT_labels, Pred_labels) * 100, 2)
        recall = round(recall_score(GT_labels, Pred_labels) * 100, 2)
        f1 = round(f1_score(GT_labels, Pred_labels) * 100, 2)
        
        #print(f"Acc: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}")
        
        dict_topic2F1[topic] = f1 
    
        
        
        # What are the forgotten items (FN) 
        images_FN = []
        for idx, img in enumerate(all_images):
            label_GT = GT_labels[idx]
            label_Pred = Pred_labels[idx]
            #FN
            if label_GT == 1 and label_Pred == 0:
                images_FN.append(img) 
        
        dict_topic2FNimg[topic] = images_FN
        
    list_dict_topic2FNimg.append(dict_topic2FNimg)
    list_dict_topic2F1.append(dict_topic2F1)
        

100%|██████████| 40/40 [36:45<00:00, 55.15s/it]


In [68]:
f = open("data/lang_list_dict_topic2FNimg.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(list_dict_topic2FNimg,f)

# close file
f.close()

In [69]:
f = open("data/lang_list_dict_topic2F1.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(list_dict_topic2F1,f)

# close file
f.close()